<a href="https://colab.research.google.com/github/dgaida/llm_client/blob/master/notebooks/RAGChatbot_groq_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Chatbot using Retrieval Augmented Generation
# Uses Groq Inference Platform to run LLM online

# TODO: benötigt groq API Key in GROQ_API_KEY und HuggingFace Token in HF_TOKEN (für Embedding Modell)
# TODO: erstelle Ordner pdfs und lade ein oder mehrere PDFs in diesen Ordner hoch

# IMPORTANT: run this cell first, then a message appears that you have to restart the session.
# in the notebook click on "restart session". Afterwards run all other cells.
# DO NOT RUN all cells immediately after opening the notebook! Run the first cell only. Then the rest.

# install packages
!apt-get -qq install -y poppler-utils tesseract-ocr libmagic1
!pip install "unstructured[all-docs]" python-magic llama-index==0.12.35 llama-index-vector-stores-chroma llama-index-embeddings-huggingface chromadb pdf2image pytesseract gradio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
libmagic1 is already the newest version (1:5.41-3ubuntu0.1).
libmagic1 set to manually installed.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.11 [186 kB]
Fetched 186 kB in 0s (590 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126718 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.11_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.11) ...
Setting up poppler-utils (22.02.0-2ubuntu0.11) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━

In [1]:
# install package that lets you use Groq or OpenAI
!git clone https://github.com/dgaida/llm_client.git
%cd llm_client
!pip install .
%cd ..

Cloning into 'llm_client'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 176 (delta 80), reused 134 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (176/176), 373.94 KiB | 4.56 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/llm_client
Processing /content/llm_client
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.5 MB/s eta 0:00:00
  Created wheel for llm-client: filename=llm_client-0.1.0-py3-none-any.whl size=9612 sha256=c5ea6b7b5b87f222bba411b9aba439eeeb5eae975ed82a34c5d62ed73f121717
  Stored in directory: /tmp/pip-ephem-wheel-cache-vyb8dgx0/wheels/d0/e6/fc/4b9383b6c127c40ef39f8e988f6338269b2cc0a669deb9aea1
Successfully built llm-client
/content


In [2]:
# --- Import required libraries ---
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import gradio as gr
from llm_client import LLMClient, LLMClientAdapter

import sys
sys.path.append("/content/llm_client/notebooks")

import utils

# --- Step 1: Set up PDF file path ---
# Put your PDF files in a folder named "pdfs" in the current directory
PDF_DIR = "pdfs"

In [4]:
# --- Step 2: Load and parse PDFs with uMiner (via UnstructuredReader) ---
all_documents = utils.read_pdf_files_with_unstructured_reader(PDF_DIR)

print(f"Loaded {len(all_documents)} documents")
for doc in all_documents[:3]:
    print(doc.text[:300])  # Zeige ersten Ausschnitt

File:Informatik_BSc_Modulhandbuch.pdf


Loaded 1 documents
Modulverzeichnis

Studiengang: Informatik (Bachelor)

Module:

Algorithmen und Programmierung I

BWL I - Grundlagen

Einführung in Betriebssysteme und Rechnerarchitekturen

Mathematik I

Theoretische Informatik I

Algorithmen und Programmierung II

BWL II - Rechnungswesen

Mathematik II

Querschnitt


In [5]:
# split documents into smaller chunks for better retrieval
node_parser = SentenceSplitter(chunk_size=256, chunk_overlap=0)  # 512, 50

In [6]:
# this is the embedding model
# models can be found here: https://huggingface.co/spaces/mteb/leaderboard
embed_model = HuggingFaceEmbedding(model_name = "intfloat/e5-small-v2")
#embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [7]:
# client that uses openAI or Grog API, here groq API
client = LLMClient(api_choice = "groq")

In [8]:
# Adapter erzeugen
adapter = LLMClientAdapter(client=client)

In [9]:
# Apply settings globally - LlamaIndex uses this Settings object internally
Settings.llm = adapter  # None
Settings.embed_model = embed_model
Settings.node_parser = node_parser

In [10]:
# --- Step 4: Create vector store and index ---
index = utils.create_chromadb_vector_store_and_index(all_documents)

In [11]:
# --- Step 5: Create query engine ---
query_engine = index.as_query_engine()

In [15]:
# this function does everyting.
# takes the query of the user and embeds it into the vector space,
# does a semantic search in the vector database (compares vectors)
# returns the text of the most similar vector
# passes this text as context to the LLM together with the query
# retrieves the response of the LLM and returns it as a string
# llama_index does this all internally using the Settings object.
def chat_with_pdf(query, history=None):
    return utils.safe_query_engine_call(query_engine, query)

In [16]:
print(chat_with_pdf("Welche Module gibt es in dem Studiengang?"))

Die Studierenden wenden im Rahmen eines realitätsnahen Projekts selbstständig erworbene Methoden und Techniken an; dabei sollen sie sich in ein bestehendes Team einfügen und sich mit gesellschaftlichen sowie rechtlichen Fragestellungen auseinandersetzen.


In [17]:
# --- Step 7: Build Gradio Interface ---
# This creates a simple web UI for asking questions
chat_ui = gr.ChatInterface(
    fn=chat_with_pdf,
    title="PDF RAG Chatbot",
    description="Ask questions about the content of your PDF documents.",
    theme="default",
    examples=["What is this PDF about?", "Summarize the second section."],
)

chat_ui.launch()

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2aa30f72f56c55c4d1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
